In [1]:
import os
import logging
import numpy as np
import netCDF4

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

from datetime import datetime
from astropy.nddata import block_reduce

import torch
from torch import nn
from torch.cuda import get_device_name
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader, RandomSampler
from torch.utils.data import Dataset, TensorDataset

from tqdm import tqdm

from nf2.data.dataset import ImageDataset
from nf2.data.loader import _load_potential_field_data, RandomCoordinateSampler
from nf2.train.model import BModel, jacobian, VectorPotentialModel

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 16 from C header, got 96 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject


# ISEE

In [2]:
class nlfff:

      def __init__(self,filename):
            self.filename=filename

            nc=netCDF4.Dataset(self.filename,'r')
            self.NOAA=nc.NOAA
            self.year_month_day_time=nc.year_month_day_time
            self.project=nc.project
            self.production_date=nc.production_date
            self.version=nc.version
            self.data_doi=nc.data_doi
            self.http_link=nc.http_link
            self.Distributor=nc.Distributor
            
            nc_x=nc.variables['x']
            self.x=nc_x[:]
            print(nc_x.long_name,' unit:',nc_x.units)
            nc_y=nc.variables['y']
            self.y=nc_y[:]
            print(nc_y.long_name,' unit:',nc_y.units)
            nc_z=nc.variables['z']
            self.z=nc_z[:]
            print(nc_z.long_name,' unit:',nc_z.units)
            
            nc_bx=nc.variables['Bx']
            self.bx=nc_bx[:].transpose(2,1,0)
            print(nc_bx.long_name,' unit:',nc_bx.units)
            nc_by=nc.variables['By']
            self.by=nc_by[:].transpose(2,1,0)
            print(nc_by.long_name,' unit:',nc_by.units)
            nc_bz=nc.variables['Bz']
            self.bz=nc_bz[:].transpose(2,1,0)
            print(nc_bz.long_name,' unit:',nc_bz.units)
            
            nc_bxp=nc.variables['Bx_pot']
            self.bx_pot=nc_bxp[:].transpose(2,1,0)
            print(nc_bxp.long_name,' unit:',nc_bxp.units)
            nc_byp=nc.variables['By_pot']
            self.by_pot=nc_byp[:].transpose(2,1,0)
            print(nc_byp.long_name,' unit:',nc_byp.units)
            nc_bzp=nc.variables['Bz_pot']
            self.bz_pot=nc_bzp[:].transpose(2,1,0)
            print(nc_bzp.long_name,' unit:',nc_bzp.units)
            
      def info(self):
            self.Lx_Mm=max(self.x) - min(self.x)
            self.Ly_Mm=max(self.y) - min(self.y)
            print(f'(Lx, Ly) in Mm = ({self.Lx_Mm:.2f}, {self.Ly_Mm:.2f})\n')
            print(f"NOAA",self.NOAA)
            print(f'year_month_day_time',self.year_month_day_time)
            print(f"project",self.project)
            print(f"production_date",self.production_date)
            print(f"version",self.version)
            print(f"data_doi",self.data_doi)
            print(f"http_link",self.http_link)
            print(f"Distributor",self.Distributor)

      def plot(self):
            xs=12.0
            ys=4.0

            xmin=min(self.x)
            xmax=max(self.x)
            ymin=min(self.y)
            ymax=max(self.y)

            plt.close()
            fig=plt.figure(figsize=(xs,ys))
            ax1=fig.add_axes((0.08,0.35,0.25,0.25*xs/ys*(ymax-ymin)/(xmax-xmin)))
            ax2=fig.add_axes((0.4,0.35,0.25,0.25*xs/ys*(ymax-ymin)/(xmax-xmin)))
            ax3=fig.add_axes((0.72,0.35,0.25,0.25*xs/ys*(ymax-ymin)/(xmax-xmin)))
            cax1=fig.add_axes((0.08,0.15,0.25,0.05))
            cax2=fig.add_axes((0.4,0.15,0.25,0.05))
            cax3=fig.add_axes((0.72,0.15,0.25,0.05))
            
            vmin=-3000.0 
            vmax=3000.0
            
            im1=ax1.pcolormesh(self.x,self.y,self.bx[:,:,0].transpose(),vmin=vmin,vmax=vmax,cmap='gist_gray',shading='auto')
            im2=ax2.pcolormesh(self.x,self.y,self.by[:,:,0].transpose(),vmin=vmin,vmax=vmax,cmap='gist_gray',shading='auto')
            im3=ax3.pcolormesh(self.x,self.y,self.bz[:,:,0].transpose(),vmin=vmin,vmax=vmax,cmap='gist_gray',shading='auto')

            cbar1=plt.colorbar(im1,cax=cax1,orientation='horizontal')
            cbar2=plt.colorbar(im2,cax=cax2,orientation='horizontal')
            cbar3=plt.colorbar(im3,cax=cax3,orientation='horizontal')
            
            ax1.set_title('Bx [G]')
            ax1.set_xlabel('x [Mm]')
            ax1.set_ylabel('y [Mm]')
            
            ax2.set_title('By [G]')
            ax2.set_xlabel('x [Mm]')
            ax2.set_ylabel('y [Mm]')
            
            ax3.set_title('Bz [G]')
            ax3.set_xlabel('x [Mm]')
            ax3.set_ylabel('y [Mm]')
            
            #plt.pause(0.1)
            # plt.savefig('./B.png')

In [3]:
# nc_filepath = '/home/tensor/workspace/pinn_study/_data/12673_20170905_202400/12673_20170905_202400.nc'
nc_filepath = '/Users/mgjeon/Workspace/pinn_study/_data/12673_20170905_202400.nc'
data = nlfff(nc_filepath)

x (westward)  unit: Mm
y (northward)  unit: Mm
z (out ot photosphere)  unit: Mm
Bx (westward)  unit: G
By (northward)  unit: G
Bz (out of photosphere)  unit: G
Bx_pot (westward)  unit: G
By_pot (northward)  unit: G
Bz_pot (out of photosphere)  unit: G


In [4]:
Lx = max(data.x) - min(data.x)
Ly = max(data.y) - min(data.y)
Lz = max(data.z) - min(data.z)

Nx = len(data.x)
Ny = len(data.y)
Nz = len(data.z)

dx = np.diff(data.x)[0]
dy = np.diff(data.y)[0]
dz = np.diff(data.z)[0]

print(f'Lx: {Lx}')
print(f'Ly: {Ly}')
print(f'Lz: {Lz}')

print(f'Nx: {Nx}')
print(f'Ny: {Ny}')
print(f'Nz: {Nz}')

# Mm per pixel
print(f'dx: {dx}')
print(f'dy: {dy}')
print(f'dz: {dz}')

# Mm^3 per pixel^3
dV = dx*dy*dz 
# cm^3 per pixel^3
dV = dx*dy*dz*(1e8**3)
print(f'dV: {dV}')

# (Nx-1) : # of pixels in x-direction
# (Nx-1)*dx : Mm in x-direction
print(f'(Nx-1)*dx: {(Nx-1)*dx}')
print(f'(Ny-1)*dy: {(Ny-1)*dy}')
print(f'(Nz-1)*dz: {(Nz-1)*dz}')

Lx: 250.724242944
Ly: 163.262298624
Lz: 163.262298624
Nx: 513
Ny: 257
Nz: 257
dx: 0.48969578700000227
dy: 0.6377433539999942
dz: 0.637743354
dV: 1.9916739845722572e+23
(Nx-1)*dx: 250.72424294400116
(Ny-1)*dy: 163.2622986239985
(Nz-1)*dz: 163.262298624


# NF2

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

# JSON
base_path = '/Users/mgjeon/Workspace/pinn_study/_run/isee_spinn'
meta_path = None

# d_slice = None
bin = 1

height = 257
spatial_norm = 320
b_norm = 2500

meta_info = None
dim = 256
positional_encoding = False
use_potential_boundary = True
potential_strides = 1
use_vector_potential = False
lambda_div = 1e-1
lambda_ff = 1e-1
decay_iterations = 50000
device = None
work_directory = None

total_iterations = 100000
batch_size = 10000
log_interval = 10000
validation_interval = 10000
num_workers = 4

# init logging
os.makedirs(base_path, exist_ok=True)
log = logging.getLogger()
log.setLevel(logging.INFO)
for hdlr in log.handlers[:]:  # remove all old handlers
    log.removeHandler(hdlr)
log.addHandler(logging.FileHandler("{0}/{1}.log".format(base_path, "info_log")))  # set the new file handler
log.addHandler(logging.StreamHandler())  # set the new console handler

start_time = datetime.now()
base_path = os.path.join(base_path, 'dim%d_bin%d_pf%s_ld%s_lf%s' % (
        dim, bin, str(use_potential_boundary), lambda_div, lambda_ff))

b_cube = np.array(np.stack([data.bx[:, :, 0], data.by[:, :, 0], data.bz[:, :, 0]], axis=-1))
meta_info = None

# if d_slice is not None:
#     b_cube = b_cube[d_slice[0]:d_slice[1], d_slice[2]:d_slice[3]]

# if bin > 1:
#     b_cube = block_reduce(b_cube, (bin, bin, 1), np.mean)

In [6]:
mf_coords = np.stack(np.mgrid[:b_cube.shape[0], :b_cube.shape[1], :1], -1)
mf_coords = mf_coords.reshape((-1, 3))
mf_values = b_cube.reshape((-1, 3))

In [7]:
class PotentialModel(nn.Module):

    def __init__(self, b_n, r_p):
        super().__init__()
        self.register_buffer('b_n', b_n)
        self.register_buffer('r_p', r_p)
        c = np.zeros((1, 3))
        c[:, 2] = (1 / np.sqrt(2 * np.pi))
        c = torch.tensor(c, dtype=torch.float32, )
        self.register_buffer('c', c)

    def forward(self, coord):
        v1 = self.b_n[:, None]
        v2 = 2 * np.pi * ((-self.r_p[:, None] + coord[None, :] + self.c[None]) ** 2).sum(-1) ** 0.5
        potential = torch.sum(v1 / v2, dim=0)
        return potential

In [8]:
pf_batch_size = int(1024 * 512 ** 2 / np.prod(b_cube.shape[:2]))

b_n = b_cube[:, :, 2]
cube_shape = (*b_n.shape, height)
b_n = b_n.reshape((-1)).astype(np.float32)
coords = [np.stack(np.mgrid[:cube_shape[0], :cube_shape[1], cube_shape[2] - 2:cube_shape[2] + 1], -1),
            np.stack(np.mgrid[:cube_shape[0], -1:2, :cube_shape[2]], -1),
            np.stack(np.mgrid[:cube_shape[0], cube_shape[1] - 2:cube_shape[1] + 1, :cube_shape[2]], -1),
            np.stack(np.mgrid[-1:2, :cube_shape[1], :cube_shape[2]], -1),
            np.stack(np.mgrid[cube_shape[0] - 2:cube_shape[0] + 1, :cube_shape[1], :cube_shape[2]], -1), ]
coords_shape = [c.shape[:-1] for c in coords]
flat_coords = np.concatenate([c.reshape(((-1, 3))) for c in coords])

r_p = np.stack(np.mgrid[:cube_shape[0], :cube_shape[1], :1], -1).reshape((-1, 3))

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
with torch.no_grad():
    b_n = torch.tensor(b_n, dtype=torch.float32, )
    r_p = torch.tensor(r_p, dtype=torch.float32, )
    model = nn.DataParallel(PotentialModel(b_n, r_p, )).to(device)

    flat_coords = torch.tensor(flat_coords, dtype=torch.float32, )

    potential = []
    for coord, in tqdm(DataLoader(TensorDataset(flat_coords), batch_size=batch_size, num_workers=2),
                        desc='Potential Boundary'):
        coord = coord.to(device)
        p_batch = model(coord)
        potential += [p_batch.cpu()]

potential = torch.cat(potential).numpy()
idx = 0
fields = []
for s in coords_shape:
    p = potential[idx:idx + np.prod(s)].reshape(s)
    b = - 1 * np.stack(np.gradient(p, axis=[0, 1, 2], edge_order=2), axis=-1)
    fields += [b]
    idx += np.prod(s)

fields = [fields[0][:, :, 1].reshape((-1, 3)),
            fields[1][:, 1, :].reshape((-1, 3)), fields[2][:, 1, :].reshape((-1, 3)),
            fields[3][1, :, :].reshape((-1, 3)), fields[4][1, :, :].reshape((-1, 3))]
coords = [coords[0][:, :, 1].reshape((-1, 3)),
            coords[1][:, 1, :].reshape((-1, 3)), coords[2][:, 1, :].reshape((-1, 3)),
            coords[3][1, :, :].reshape((-1, 3)), coords[4][1, :, :].reshape((-1, 3))]

pf_coords, pf_values = np.concatenate(coords), np.concatenate(fields)

Potential Boundary:   0%|          | 0/159 [01:04<?, ?it/s]


KeyboardInterrupt: 

In [41]:
coords = np.stack(np.mgrid[:cube_shape[0]:1, :cube_shape[1]:1, :cube_shape[2]:1], -1)

In [35]:
coords.shape

(513, 257, 257, 3)

In [48]:
coords

(3, 257, 257, 3)